In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from PMFG import PMFG

In [ ]:
raw_asset_prices_df = pd.read_csv("IVV_historical.csv", index_col='Date')
log_returns_df = np.log(raw_asset_prices_df).diff().dropna()
# drop first row of raw prices so it has the same dimensions as the log-returns DF
raw_asset_prices_df = raw_asset_prices_df.iloc[1:]

df_shape = (raw_asset_prices_df.shape)
print(f"There are {df_shape[0]} rows and {df_shape[1]} columns in the dataset.")
print(f"Data timeperiod covers: {raw_asset_prices_df.index[0]} to {raw_asset_prices_df.index[-1]}")

raw_asset_prices_df.head()

In [ ]:
raw_corr = log_returns_df.corr()

shr_coef = 1e-4
#shr_target=np.ones((df_shape[1], df_shape[1]))
shr_target=np.eye(df_shape[1])

correlation_matrix = raw_corr*(1-shr_coef) + shr_target*shr_coef
print('Condition number of sample correlation matrix: %.2e' %np.linalg.cond(raw_corr))
print('Condition number of shrunk correlation matrix: %.2e' %np.linalg.cond(correlation_matrix))
correlation_matrix.head()

In [ ]:
G = nx.Graph()
nodes_list = correlation_matrix.columns.values.tolist()

for i in range(len(nodes_list)):
    G.add_node(nodes_list[i])

for i in range(len(nodes_list)):
    for j in range(i+1, len(nodes_list)):
        G.add_edge(nodes_list[i], nodes_list[j], weight=correlation_matrix.iloc[i,j])

In [ ]:
test = PMFG(G)
test.compute()